In [1]:
import pandas as pd
import pymongo

db_client = "mongodb://localhost:27017/"
db_name = "datascience_presentation_1"

# Define required functions


def query_mdb(collection: str, query: dict = {}, print_query: bool = True):
    '''
    Function to make database queries on my mongodb.
    '''
    return query_db(db_client,
                    db_name,
                    collection,
                    query,
                    print_query)


def query_db(client, database: str, collection: str, query: dict, print_query: bool):
    '''
    Function to make database queries on a database collection in the given mongodb.
    '''
    if print_query:
        print(query)

    items = pd.DataFrame()

    client_ = pymongo.MongoClient(client)
    database_ = client_[database]
    collection = database_[collection]

    cursor = collection.find(query)
    items = pd.DataFrame(cursor)

    return items


def get_mdb():
    client = pymongo.MongoClient(db_client)
    database = client[db_name]
    return database


db = get_mdb()

In [2]:
# INFO: Falls diese Einträge schon vorhanden sind, kann man via dem MongoDB Compass die Collection "texts" einfach droppen und
# von Null anfangen.

# insert base data
books = [{
    "name": "Testbuch 1",
    "author": "Mighty Chad",
    "pages": "101",
    "genres": [
        "fantasy",
        "scify",
        "romance"
    ]
},
    {
    "name": "Testbuch 2",
    "author": "Mighty Giga",
    "pages": "202",
    "genres": [
        "novel",
        "drama"
    ]
},
    {
    "name": "Testbuch 3",
    "author": "Mighty Prompt",
    "pages": "303",
    "genres": [
        "fantasy",
        "horror"
    ]
},
    {
    "name": "Testbuch 0",
    "author": "Mighty Zero",
    "pages": "1",
    "genres": [
        "romance"
    ]
}]

get_mdb()["texts"].insert_many(books)

# Falls man selber eigene hinzufügen will:
# item_1 = {
#     "name": "Testbuch 0",
#     "author": "Mighty Zero",
#     "pages": "1",
#     "genres": [
#         "romance"
#     ]
# }

# get_mdb()["texts"].insert_one(item_1)

InsertManyResult([ObjectId('6603b90777190b4fac1aca90'), ObjectId('6603b90777190b4fac1aca91'), ObjectId('6603b90777190b4fac1aca92'), ObjectId('6603b90777190b4fac1aca93')], acknowledged=True)

# MongoDB

## Data modeling

### No Joins

-   MongoDB unterstützt keine typische JOIN Funktion wie SQL.
    -   Liegt daran, dass MongoDB dokument-orientiert ist und für das abfragen vieler großer Dokumente optimiert wurde.
    -   Daraus folgt, dass man seine Kollektionen (Collections) so entwerfen muss, dass sie bei einer Abfrage alle
        benötigten weiteren Informationen verschachtelt enthalten.
        -   Dadurch spiegelt das Datenbank-Design recht starkt die Verschiedenen Anwendungsfälle einer Anwendung wider.
        -   Dies sorgt aber auch für einfachere und weniger Datenbank-Anfragen, da man dann nicht erst alle benötigten
            Informationen zusammen orchestrieren muss.
            -> Will man eine Informations-Seite zu einem Buch anzeigen, so sind im Datenbank Buch-Objekt schon
            Autor-Name, ISBN und Seitenzahl etc. enthalten.
-   Aber in Version 3.2 kam der `$lookup` Operator hinzu, mit dem man einen "Left Outer Join" durchführen kann.
    -   Dieser ist aber nicht so performant wie bei einer SQL Datenbank und sollte nicht als Ersatz dafür verwendet werden.
-   Besser das Daten-Schema gut entwerfen und an den Anwendungs Abfrage-Mustern orienteren.
-   Und bei Änderungen der Anwendung das Schema an diese Änderungen mit anpassen.

### Arrays

-   MongoDB unterstützt Arrays und bietet eine Vielzahl von Operatoren und Methoden, um mit Arrays zu arbeiten.


In [8]:
query_mdb("texts")

{}


,_id,name,author,pages,genres
0,6603b90777190b4fac1aca90,Testbuch 1,Mighty Chad,101,"[fantasy, scify, romance]"
1,6603b90777190b4fac1aca91,Testbuch 2,Mighty Giga,202,"[novel, drama]"
2,6603b90777190b4fac1aca92,Testbuch 3,Mighty Prompt,303,"[fantasy, horror]"
3,6603b90777190b4fac1aca93,Testbuch 0,Mighty Zero,1,[romance]


In [4]:
query_mdb("texts", {"genres": {"$all": ["fantasy"]}})

{'genres': {'$all': ['fantasy']}}


,_id,name,author,pages,genres
0,6603b90777190b4fac1aca90,Testbuch 1,Mighty Chad,101,"[fantasy, scify, romance]"
1,6603b90777190b4fac1aca92,Testbuch 3,Mighty Prompt,303,"[fantasy, horror]"


In [9]:
query_mdb("texts", {"genres": {"$all": ["fantasy", "scify"]}})

{'genres': {'$all': ['fantasy', 'scify']}}


,_id,name,author,pages,genres
0,6603b90777190b4fac1aca90,Testbuch 1,Mighty Chad,101,"[fantasy, scify, romance]"


-   Der Operator `$all` kann verwendet um Dokumente mit einem Array zu finden, die alle gewünschten Elemente im Array
    eines Schlüssels haben.
    -   Der Schlüssel `genres` muss bei einem Dokument also zum Beispiel `fantasy` und `horror` enthalten um gefunden zu werden.
-   Operator `$elemMatch` findet Dokumente bei denen ein Element in einem Array eines Schlüssels in allen Feldern mit
    den angegebenen übereinstimmt.
    -   Beispiel aus der Doku: The following query matches documents where results contains at least one element where product is "xyz" and score is greater than or equal to 8:

```js
db.survey.insertMany( [
   { "_id": 1, "results": [ { "product": "abc", "score": 10 },
                            { "product": "xyz", "score": 5 } ] },
   { "_id": 2, "results": [ { "product": "abc", "score": 8 },
                            { "product": "xyz", "score": 7 } ] },
   { "_id": 3, "results": [ { "product": "abc", "score": 7 },
                            { "product": "xyz", "score": 8 } ] },
   { "_id": 4, "results": [ { "product": "abc", "score": 7 },
                            { "product": "def", "score": 8 } ] },
   { "_id": 5, "results": { "product": "xyz", "score": 7 } }
] )

db.survey.find(
    { results: { $elemMatch: { product: "xyz", score: { $gte: 8 } } } }
)

{ "_id" : 3, "results" : [ { "product" : "abc", "score" : 7 },
                    { "product" : "xyz", "score" : 8 } ] }
```

-   Operator `$size` selects documents if the array field is a specified size.


In [6]:
query_mdb("texts", {"genres": {"$size": 1}})

{'genres': {'$size': 1}}


,_id,name,author,pages,genres
0,6603b90777190b4fac1aca93,Testbuch 0,Mighty Zero,1,[romance]


In [7]:
query_mdb("texts", {"genres": {"$size": 3}})

{'genres': {'$size': 3}}


,_id,name,author,pages,genres
0,6603b90777190b4fac1aca90,Testbuch 1,Mighty Chad,101,"[fantasy, scify, romance]"


-   Operator `$push` aktualisiert einen Eintrag in einem Array und fügt ein neues Element in diesen Array ein.

```js
// create example item
db.students.insertOne( { _id: 1, scores: [ 44, 78, 38, 80 ] } );

// javascript command to do the '$push'
db.students.updateOne({ _id: 1 }, { $push: { scores: 89 } });

// result
{ _id: 1, scores: [ 44, 78, 38, 80, 89 ] }

```

-   Operator `$addToSet`:
    -   If the value is an array, `$addToSet` appends the whole array as a single element.

```js
db.alphabet.insertOne( { _id: 1, letters: ["a", "b"] } )

db.alphabet.updateOne(
   { _id: 1 },
   { $addToSet: { letters: [ "c", "d" ] } }
)

{ _id: 1, letters: [ 'a', 'b', [ 'c', 'd' ] ] }
```

-   Operator `$pop`:
    -   The following example removes the first element, 8, from the scores array:

```js
db.students.insertOne( { _id: 1, scores: [ 8, 9, 10 ] } )

db.students.updateOne( { _id: 1 }, { $pop: { scores: -1 } } )

{ _id: 1, scores: [ 9, 10 ] }
```

-   Operator `$pull`:
    -   The following operation will remove all items from the votes array that are greater than or equal to ( `$gte` ) 6:

```js
db.profiles.insertOne( { _id: 1, votes: [ 3, 5, 6, 7, 7, 8 ] } )

db.profiles.updateOne( { _id: 1 }, { $pull: { votes: { $gte: 6 } } } )

{ _id: 1, votes: [  3,  5 ] }
```

-   Operator `$each`:
    -   Then the following operation uses the $addToSet operator with the `$each` modifier to add multiple elements to the
        tags array:

```js
{ _id: 2, item: "cable", tags: [ "electronics", "supplies" ] }

db.inventory.updateOne(
   { _id: 2 },
   { $addToSet: { tags: { $each: [ "camera", "electronics", "accessories" ] } } }
 )

{
  _id: 2,
  item: "cable",
  tags: [ "electronics", "supplies", "camera", "accessories" ]
}
```

-   Es gibt noch weitere Operatoren: `$pullAll`, `$position`, `$slice`, `$sort` die
    ähnlich funktionieren und Werte in Arrays verändern.
    -   Siehe: https://www.mongodb.com/docs/manual/reference/operator/update-array/


### Embedded Documents

-   Sind verschachtelte Elemente in anderen Dokumenten. Beispiel:

```js
{
 _id: ObjectId('507f191e810c19729de860ea'),
 product_name: 'Coffee Mug',
 manufacturer: {
    name: 'Good Coffee Inc.',
    location: 'Portland, Oregon'
 }
}
```

-   Hier ist `manufacturer` ein eingebettetes/verschachteltes Dokument in einem Produkt Haupt-Dokument welches einen Kaffebecher repräsentiert.
-   Bilden eine hirarchische Datenstruktur innerhalb der Dokumente.
-   Vorteile:
    -   Sie ermöglichen atomare Operationen, reduzieren die Notwendigkeit separater Abfragen und vermeiden Joins, die in MongoDB nicht nativ unterstützt werden.
-   Nachteile:
    -   Sie können jedoch auch zu einem Wachstum der Dokumente führen, was zu Leistungsproblemen führen kann, und sind auf die BSON-Dokumentgrößenbeschränkung von 16 MB begrenzt 1.

## Aggregation Data

### Aggregation Pipeline

-   Eine Aggregation Pipeline in MongoDB ist eine Reihe von Operationen, die auf Dokumente in einer Sammlung angewendet
    werden, um Daten zu verarbeiten, zu transformieren und Ergebnisse zu liefern. Die Pipeline besteht aus mehreren
    Stufen, die nacheinander ausgeführt werden. Jede Stufe führt eine Operation auf den Eingabedokumenten durch, wie
    z.B. Filtern, Gruppieren oder Berechnungen durchführen. Die Dokumente, die von einer Stufe ausgegeben werden, werden
    an die nächste Stufe weitergeleitet. Die Aggregation Pipeline kann Ergebnisse für Gruppen von Dokumenten liefern,
    z.B. Gesamt-, Durchschnitts-, Maximal- und Minimalwerte. Ab MongoDB 4.2 können Dokumente mit einer Aggregation
    Pipeline aktualisiert werden, wenn die entsprechenden Stufen verwendet werden. Beispiel:

```js
db.orders.aggregate([
    // Stage 1: Filter pizza order documents by pizza size
    {
        $match: { size: "medium" },
    },
    // Stage 2: Group remaining documents by pizza name and calculate total quantity
    {
        $group: { _id: "$name", totalQuantity: { $sum: "$quantity" } },
    },
]);
```

-   Diese Pipeline besteht aus zwei Stufen: $match filtert die Dokumente nach Größe und $group gruppiert die verbleibenden Dokumente nach Pizzanamen und berechnet die Gesamtmenge

-   Die Aggregation Pipeline ist oft die bevorzugte und empfohlene Methode für Aggregationen in MongoDB, da sie speziell für eine verbesserte Leistung und Benutzerfreundlichkeit konzipiert ist. Sie kann neue Dokumente generieren oder Dokumente filtern, und ab MongoDB Version 4.4 können benutzerdefinierte Aggregationsausdrücke mit $accumulator und $function definiert werden.
-   Die Aggregation Pipeline bietet eine Vielzahl von Operatoren, die in verschiedenen Stufen der Pipeline verwendet werden können, um Ausdrücke zu konstruieren. Diese Operatoren umfassen arithmetische, Array-, boolesche, Vergleichs-, bedingte, benutzerdefinierte Aggregations-, Datensatzgrößen-, Datums-, Literal-, Objekt-, Set-, String-, Text-, Trigonometrie- und Typoperatoren.

### Indexes

-   Indexes in MongoDB sind spezielle Datenstrukturen, die einen kleinen Teil des Datensatzes einer Sammlung in einer leicht durchsuchbaren Form speichern. Sie ermöglichen die effiziente Ausführung von Abfragen, indem sie MongoDB die Anzahl der zu überprüfenden Dokumente begrenzen, wenn ein geeigneter Index für eine Abfrage vorhanden ist. Ohne Indexe muss MongoDB jedes Dokument in einer Sammlung scannen, um diejenigen auszuwählen, die der Abfrageanweisung entsprechen, was bei großen Sammlungen zeitaufwendig und ressourcenintensiv ist.
-   MongoDB bietet eine breite Palette an Indextypen und Funktionen, darunter:
    -   Einzel-Feld-Indexe: Diese Indexe werden auf ein einzelnes Feld eines Dokuments erstellt. MongoDB erstellt automatisch einen Index auf dem \_id-Feld, aber zusätzliche einzelne Feld-Indexe können auf anderen Feldern erstellt werden.
    -   Zusammengesetzte Indexe: Diese Indexe werden auf mehrere Felder eines Dokuments erstellt. Die Reihenfolge der Felder in einem zusammengesetzten Index ist signifikant und beeinflusst, wie der Index Abfrageoperationen unterstützen kann.
    -   Multi-Key-Indexe: Diese Indexe werden verwendet, um den Zugriff auf einzelne Elemente in Arrays zu ermöglichen. MongoDB erstellt für jedes Element im Array einen Indexschlüssel.
    -   Text-Indexe: Für Daten, die auf MongoDB Atlas gehostet werden, bietet Atlas Full Text Search einen vollständig verwalteten Lucene-Index, der in die MongoDB-Datenbank integriert ist.
    -   Geografische Indexe: MongoDB bietet 2d-Indexe für planare Geometrie und 2dsphere-Indexe für sphärische Geometrie.
    -   Hashed Indexe: Diese Indexe werden verwendet, um Hash-basiertes Sharding zu unterstützen, indem der Hash des Wertes eines Feldes indiziert wird.
    -   TTL (Time-To-Live) Indexe: Diese speziellen Indexe können automatisch Dokumente aus einer Sammlung entfernen, nachdem eine bestimmte Zeit verstrichen ist.
-   Indexe können bei Bedarf erstellt und gelöscht werden, um sich ändernden Anwendungsanforderungen und Abfragemustern gerecht zu werden. Sie können für jedes Feld in Ihren Dokumenten deklariert werden, einschließlich der in Arrays verschachtelten Felder.
-   Es ist wichtig zu beachten, dass Indexe ressourcenintensiv sind und zusätzlichen Speicherplatz verbrauchen. Während Felder aktualisiert werden, muss der zugehörige Index beibehalten werden, was zusätzlichen CPU- und Festplatten-E/A-Overhead verursacht. MongoDB bietet Tools, die Ihnen helfen, die Indexnutzung zu verstehen und zu optimieren.
-   Für eine effiziente Nutzung von Indexen in MongoDB ist es wichtig, die richtige Strategie zu wählen, um die Leistung zu optimieren. Dies beinhaltet die Verwendung von abgedeckten Abfragen, bei denen Ergebnisse direkt aus einem Index geliefert werden, ohne dass auf die Quelldokumente zugegriffen werden muss, und die Nutzung des Explain-Plans, um die Indexabdeckung für einzelne Abfragen zu überprüfen.

### Explain

-   In MongoDB ist explain() eine Funktion, die verwendet wird, um Informationen über die Ausführung einer Abfrage zu erhalten. Sie gibt detaillierte Informationen über den Abfrageplan und die Ausführungsstatistiken zurück, was hilft, die Leistung von Abfragen zu analysieren und zu optimieren.
    -   Abfrageplan: explain() zeigt den von MongoDB verwendeten Abfrageplan an, einschließlich der verwendeten Indizes und der Strategie zur Durchführung der Abfrage (z.B. COLLSCAN für eine vollständige Sammlungsüberprüfung oder IXSCAN für einen Index-Scan).
    -   Ausführungsstatistiken: Durch Anhängen von "executionStats" an explain() erhalten Sie zusätzliche Informationen wie die Anzahl der untersuchten Dokumente und Schlüssel, die Gesamtzeit der Ausführung und spezifische Details zu den Ausführungsstufen der Abfrage.
-   Ein Beispiel für die Verwendung von explain() mit Ausführungsstatistiken sieht folgendermaßen aus:

```js
db.users.find({ username: "USER_999999" }).explain("executionStats")
    .executionStats;
```

-   Diese Funktion ist besonders nützlich, um zu verstehen, wie MongoDB Abfragen ausführt und um Bereiche zu identifizieren, die möglicherweise optimiert werden können, z.B. durch Hinzufügen oder Anpassen von Indizes.

### Import

-   `Import` in MongoDB bezieht sich auf den Prozess, Daten aus externen Quellen in eine MongoDB-Datenbank zu übertragen. MongoDB bietet verschiedene Methoden zum Importieren von Daten, darunter das mongoimport Tool, das speziell für diesen Zweck entwickelt wurde. Mit mongoimport können Benutzer Daten aus CSV-Dateien, JSON-Dateien oder anderen unterstützten Formaten direkt in eine MongoDB-Datenbank importieren. Dieser Prozess ist besonders nützlich für die Initialisierung einer Datenbank mit vorhandenen Daten oder für die Aktualisierung bestehender Datenmengen.

### Export

-   `Export` in MongoDB bezieht sich auf den Prozess, Daten aus einer MongoDB-Datenbank in externe Formate wie JSON oder CSV zu übertragen. Dieser Prozess wird mit dem Befehl mongoexport durchgeführt, einem Befehlszeilentool, das speziell für diesen Zweck entwickelt wurde. Mit mongoexport können Benutzer Daten aus einer MongoDB-Datenbank exportieren und in einer Datei speichern, die dann für andere Zwecke verwendet werden kann, wie z.B. für Backups, Datenanalyse oder den Import in eine andere Datenbank.
-   Ein einfaches Beispiel für den Export einer Sammlung in JSON-Format wäre:

```bash
mongoexport --db=sales --collection=contacts --out=contacts.json
```

-   Und für den Export in CSV-Format, wobei spezifische Felder ausgewählt werden:

```bash
mongoexport --db=users --collection=contacts --type=csv --fields=name,address --out=/opt/backups/contacts.csv
```
